<a href="https://colab.research.google.com/github/SoumojjalSen/RAG-Chatbot/blob/main/RAG_chatbot_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RAG System Using Llama2 With Hugging Face

In [ ]:
!pip install -q pypdf;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.5 MB/s eta 0:00:00


In [ ]:
## Embedding
!pip install -q sentence_transformers;

In [ ]:
!pip install -q llama_index;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.6/250.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install -q llama-index-llms-huggingface;

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

* VectorStoreIndex: This typically represents an index that stores vector embeddings of text data. It enables fast similarity searches, so you can quickly find relevant content based on context or keywords by comparing vector embeddings.

* SimpleDirectoryReader: This reads files from a specified directory. It helps in loading multiple files for processing, such as documents you want to index or analyze.

* ServiceContext: This provides a shared context or configuration for services interacting with the index. It often manages settings like API keys, logging, or other resources that services need.

In [ ]:
ls

sample_data/


In [11]:
documents = SimpleDirectoryReader("sample_data", required_exts=[".pdf"]).load_data()

In [12]:
documents

[Document(id_='50945990-a7d5-45bb-b789-b44424eeb09f', embedding=None, metadata={'page_label': '1', 'file_name': 'lcs_pdf.pdf', 'file_path': '/content/sample_data/lcs_pdf.pdf', 'file_type': 'application/pdf', 'file_size': 55045, 'creation_date': '2025-01-30', 'last_modified_date': '2025-01-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Longest Common Subsequence\nA subsequence of a string S, is a set of characters that appear in left-\nto-right order, but not necessarily consecutively.\nExample\nACT T GCG\n• ACT , AT T C, T , ACT T GCare all subsequences.\n• T T Ais not a subequence\nA common subequence of two strings is a subsequenc

In [14]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login

In [16]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.2, "do_sample": False}, # temperature tells how creative the LLM model should be
    system_prompt=system_prompt, # system prompt we have created
    query_wrapper_prompt=query_wrapper_prompt, # query wrapper prompt we have created
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf", # llama model with 7 billion parameter
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
%pip install -q llama-index-embeddings-langchain;

In [ ]:
!pip install -q langchain-community;

In [19]:
## The entire model has been uploaded

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    )

# This is a sentence-transformers model: It maps sentences & paragraphs to a 768 dimensional dense vector space and can be used for tasks like clustering or semantic search.

<ipython-input-19-4c0a91646789>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Now, I have Embed model, LLM model and documents... So, we are going to combine all of them using ServiceContext. ServiceContext bundles all the indexes and queries.

But as ServiceContext is deprecated, we use Settings

In [20]:
from llama_index.core import Settings

Settings.chunk_size = 1024
Settings.llm = llm
Settings.embed_model = embed_model

In [21]:
index=VectorStoreIndex.from_documents(documents)

In [22]:
index

In [23]:
query_engine=index.as_query_engine()

In [24]:
response=query_engine.query("What is LCS problem and write the code in C++ to calculate the LCS taking sample strings as input")

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [25]:
print(response)


The LCS problem is a well-known problem in computer science and bioinformatics that involves finding the longest common subsequence (LCS) between two or more strings. The LCS is the longest sequence of symbols that is common to all the given strings.

To calculate the LCS in C++, you can use the following algorithm:
```
#include <iostream>
#include <string>
#include <vector>

using namespace std;

// Function to calculate the LCS of two strings
vector<vector<int>> lcs(string x, string y) {
    // Initialize two vectors to store the LCS
    vector<int> c(x.size() + 1, 0);
    vector<int> b(y.size() + 1, 0);

    // Initialize two pointers to iterate over the strings
    int i = 0;
    int j = 0;

    // Loop until one of the pointers reaches the end of the string
    while (i < x.size() && j < y.size()) {
        // Compare the current characters of the two strings
        if (x[i] == y[j]) {
            // If the characters are the same, increment the LCS
            c[i] = max(c[i], 